In [4]:
import pandas as pd

# 母型船资料

In [11]:
# 海水密度
rho = 1.025 # 单位: t/m^3
# 母船主尺度信息
L_total_parent = 225 # 母船总长度
Lpp_parent = 218 # 母船垂线间长
B_parent = 32.26 # 母船型宽
D_parent = 19.6 # 母船型深
d_design_parent = 12.5 # 母船设计吃水
d_structure_parent = 14.3 # 母船结构吃水\
DW_parent = 75088 # 母船载重量 :t
Lw_parent = Lpp_parent+5*0.75 # 水线间长:m ,首垂线与尾垂线距离+ 尾垂线后5个肋位，每个肋位距离为0.75m，参考母船总布置图
L_ball_parent = 4*0.75 # 首垂线至球首前端距，#256 - # 260，肋距为0.75m，参考母船总布置图
nabla_parent = 85488.56 # 母船结构吃水14.3m处的排水体积，参考静水力计算书
delta_parent = 87786.17 # 母船结构吃水处的排水量,参考任务书第八章母船数据
# 母船肋位信息
costal1_range_parent = [-5,11] # 肋位：[开始肋位,结束肋位]
costal1_span_parent = 750 # 肋距，单位 mm
costal2_range_parent = [11,36]
costal2_span_parent = 800 # mm
costal3_range_parent = [36,242]
costal3_span_parent = 870 # mm
costal4_range_parent = [242,260]
costal4_span_parent = 750 # mm
# 设计船信息
DW = 72100 # 设计船载重量,t

In [12]:
pship_info = pd.Series({
    'L':L_total_parent,
    'Lpp': Lpp_parent,
    'B': B_parent,
    'D': D_parent,
    'design d': d_design_parent,
    'structure d': d_structure_parent,
    'DW': DW_parent,
    '水线长': Lw_parent,
    '首垂线到球首的距离': L_ball_parent,
    '结构吃水处的排水量': delta_parent
})
print("母船数据表:")
pship_info

母船数据表:


L                225.00
Lpp              218.00
B                 32.26
D                 19.60
design d          12.50
structure d       14.30
DW             75088.00
水线长              221.75
首垂线到球首的距离          3.00
结构吃水处的排水量      87786.17
dtype: float64

# 设计船资料
**设计一艘载重量72100t的散货船**，参考母型船，设计船的肋位数据如下

In [33]:
costal1_range = [-5,11] 
costal1_span = 750 # 肋距，毫米
costal2_range = [11,36]
costal2_span = 800
costal3_range = [36,242]
costal3_span = 850
costal4_range = [242,260]
costal4_span = 750

## 设计船排水量粗估

In [14]:
# 母船载重量系数计算
eta_DW_parent = DW_parent / delta_parent
print("母船载重量系数: ",eta_DW_parent)

母船载重量系数:  0.8553511333277213


根据公式$\triangle = \frac{DW}{\eta_{DW}}$，设计船的排水量为

In [16]:
# 设计船的排水量
delta = int(DW/eta_DW_parent)
print("设计船的排水量:",delta,'t')

设计船的排水量: 84292 t


根据母船换算设计船的主尺度，有公式:  
$L=L_o(\triangle / \triangle_0)^{1/3}$  
$B=B_o(\triangle / \triangle_0)^{1/3}$  
$d=d_o(\triangle / \triangle_0)^{1/3}$  
$D=D_o\frac{d}{d_o}$  
为方便描述与计算，设系数$(\triangle / \triangle_0)^{1/3}$为$k$

In [25]:
k = (delta/delta_parent)**(1/3)
k = round(k,3)
print("k: ",k)

k:  0.987


利用母船换算公式换算设计船的主尺度，参考课本p122

In [28]:
Lpp = round(Lpp_parent*k,2)
B = round(B_parent*k,2)
d_design = round(d_design_parent*k,2)
d_structure = round(d_structure_parent*k,2)
D = round(D_parent*d_structure/d_structure_parent,2)
print("设计船垂线间长为:",Lpp,'m')
print("设计船型宽为:",B,'m')
print("设计船结构吃水和设计吃水分别为:",d_structure,'m ',d_design,'m')
print("设计船型深为:" , D , 'm')

设计船垂线间长为: 215.17 m
设计船型宽为: 31.84 m
设计船结构吃水和设计吃水分别为: 14.11 m  12.34 m
设计船型深为: 19.34 m


## 防撞舱壁设计

由球首的船，防撞舱壁起点取下面的最小值:  
+ 首垂线至球首前端长度的中点
+ 首垂线之前 0.015$L_L$处($L_L$是水线长)
+ 首垂线之前 3m 处

In [30]:
AntiCollisionBulkheadStart = min(L_ball_parent*0.5,
                                0.015*Lw_parent,
                                3)
print("防撞舱壁起点距离首垂线距离:",AntiCollisionBulkheadStart,'m')

防撞舱壁起点距离首垂线距离: 1.5 m


规范规定防撞舱壁至首垂线的距离应不小于 0.05$L_L$或 10m（取较小者），且不大于 0.08$L_L$。

In [31]:
minAntiLength = min(0.05*Lw_parent,10)
maxAntiLength =round(0.08*Lw_parent,2)
print("防撞舱壁长度范围%s m - %s m"%(minAntiLength,maxAntiLength))

防撞舱壁长度范围10 m - 17.74 m


而母船防撞舱壁取首垂线前14个肋位，在此设计船也取14个肋位

In [34]:
lengthAfterFp = 14*costal4_span/1000
lengthBeforeFp = AntiCollisionBulkheadStart
length = lengthAfterFp + lengthBeforeFp
print("防撞舱壁长度范围为%s"%length)
print("是否满足规范要求",minAntiLength<length<maxAntiLength)

防撞舱壁长度范围为12.0
是否满足规范要求 True


## 尾尖舱设计

散货船尾尖舱长度一般约为（0.038～0.045）$L_{PP}$，对于总长有限制且吨位又较大的船，从尽可能提高舱容利用率可考虑取较短的尾尖舱。

In [36]:
minLength = round(0.038*Lpp,3)
maxLength = round(0.045*Lpp,3)
print("尾尖舱长度范围为%s m-%s m"%(minLength,maxLength))

尾尖舱长度范围为8.176 m-9.683 m


此处取整数肋位长度

In [37]:
minCostalNum = round(minLength/costal1_span*1000,1)
maxCostalNum = round(maxLength/costal1_span*1000,1)
print("肋距个数在%s - %s"%(minCostalNum,maxCostalNum))

肋距个数在10.9 - 12.9


In [38]:
# 取11个肋位
tailPeakCabinLength = 11*costal1_span/1000
print("尾尖舱长度为:%s"%tailPeakCabinLength)

尾尖舱长度为:8.25


## 机舱设计

机舱长度设计比较可靠的方法还是参照母型船用比较法来选取，取24个肋位

In [40]:
# 取机舱长度为肋距的24倍
engineRoomLength = 24*costal2_span/1000
print("机舱长度为:%s"%engineRoomLength)

机舱长度为:19.2


## 货仓设计

In [42]:
# 剩余长度为
remainingLength = Lpp-tailPeakCabinLength\
            -engineRoomLength-lengthAfterFp
print("剩余长度为%s m"%remainingLength)

剩余长度为177.22 m


货仓间1槽型舱壁占据一个肋位，7个舱室，6个舱壁，总占据长度为:

In [14]:
L_bulk = 6*costal3_span/1000
L_bulk

5.1

In [15]:
# 货仓实际长度
warehouseLength = remainingLength - L_bulk
warehouseLength

172.10000000000002

In [16]:
# 是否为整肋位长度:否
warehouseLength/costal3_span*1000

202.47058823529414

In [17]:
# 为便于肋骨布置与肋位安排，此处取货仓长度为整数个肋距，取
# 货仓长度为202个肋距
warehouseLength = 202*costal3_span/1000
# 每个货舱的肋位数为
# 28 29 29 29 29 29 29

In [20]:
warehouseLength

171.7

由于货仓段缩短，设计船的垂线间长减小，其他主尺度也相应减小

In [18]:
lengthAfterFp

10.5

In [19]:
Lpp_final = round(tailPeakCabinLength + engineRoomLength +\
        lengthAfterFp + L_bulk+202*costal3_span/1000,2)
B_final = round(Lpp_final/Lpp_parent*B_parent,2)
d_design_final = round(Lpp_final/Lpp_parent*d_design_parent,2)
d_structure_final = round(Lpp_final/Lpp_parent*d_structure_parent,2)
D_final = round(Lpp_final/Lpp_parent*D_parent,2)
print('''
初步设计的尺寸为:
 垂线间长: %s m
 型宽: %s m
 设计吃水: %s m
 结构吃水: %s m
 型深: %s m
'''%(Lpp_final,B_final,d_design_final,d_structure_final,D_final))


初步设计的尺寸为:
 垂线间长: 214.75 m
 型宽: 31.78 m
 设计吃水: 12.31 m
 结构吃水: 14.09 m
 型深: 19.31 m

